Azure Cosmos DB cache is available only if you have dedicated Gateway Server.
There are different size of servers,
You need to use the dedicated gateway server's connection string and override your connection type to Gateway mode.
Integrated Cache can cache query and point read results.
You can define the expiration time of each item by code.
Integrated Cache is a great way to save R/U(money) for Read-heavy workloads (Your data doesn't change frequently)
ConsistencyLevel of the operation must be Session or Eventual for caching to work.

In [ ]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;

In [ ]:
public class Post    
{
    public string id { get; set; }
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int AnswerCount { get; set; }
    public int CommentCount { get; set; }
    public int FavoriteCount { get; set; }
    public int AcceptedAnswerId { get; set; }
    public DateTime? CreatedOn { get; set; }
    public DateTime? ClosedDate { get; set; }
    public int OwnerUserId { get; set; }
    public string OwnerDisplayName { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public string Tags { get; set; }
}

You need to run this in Azure. Local Emulator does not support the caching feature.
Paste the connection string of your dedicated gateway server to the following script to execute it.

In [ ]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(cstring, new CosmosClientOptions
{
    ConnectionMode = ConnectionMode.Gateway
});
var db = client.GetDatabase("StackOverflow");
var postContainer = db.GetContainer("Posts");

In [ ]:
var cmd = new QueryDefinition("SELECT * FROM c WHERE c.OwnerUserId = @id")
    .WithParameter("@id", "1");
var query= new QueryDefinition(cmd);
var iterator = postContainer.GetItemQueryIterator<Post>(query, 
requestOptions: new QueryRequestOptions
{
    ConsistencyLevel = ConsistencyLevel.Eventual,
    DedicatedGatewayRequestOptions = new DedicatedGatewayRequestOptions
    {
        MaxIntegratedCacheStaleness = TimeSpan.FromSeconds(30),
        BypassIntegratedCache = false        
    }
});
var results = new List<Post>();
while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    results.AddRange(response);
    response.requestCharge.Display();
}
